In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

### Test If gpt-3.5 knows Korea a bit

In [4]:
prompt = "Do you know what is needed if the foreigner wants to drive in Korea? Please, answer with less than 30 words."

get_completion(prompt)

"Foreigners need an International Driving Permit (IDP) or a valid driver's license from their home country, along with a Korean translation of the license, to drive in Korea."

In [5]:
prompt = "Is smoking in public space allowed in Korea? Please, answer with less than 30 words."

get_completion(prompt)

'No, smoking in public spaces is not allowed in Korea.'

In [6]:
prompt = "What would be illegal in Korea that foreigner may miss easily? Please explain less than 100 words."

get_completion(prompt)

'One thing that may be illegal in Korea and easily missed by foreigners is jaywalking. In Korea, it is strictly enforced and pedestrians are expected to cross the road only at designated crosswalks or pedestrian crossings. Jaywalking can result in fines or even imprisonment. It is important for foreigners to be aware of and follow local traffic laws to avoid any legal issues.'

### Below is other samples or stratagies

#### Tactic 4: "Few-shot" prompting

In [11]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \
valley flows from a modest spring; the \
grandest symphony originates from a single note; \
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: Resilience is like a mighty oak tree that withstands the strongest storms, bending but never breaking. It is the ability to bounce back from adversity, to find strength in the face of challenges, and to persevere even when the odds seem insurmountable. Just as a diamond is formed under immense pressure, resilience is forged through the trials and tribulations of life.


### Principle 2: Give the model time to “think”

#### Tactic 1: Specify the steps required to complete a task

In [12]:
text = f"""
In a charming village, siblings Jack and Jill set out on \
a quest to fetch water from a hilltop \
well. As they climbed, singing joyfully, misfortune \
struck—Jack tripped on a stone and tumbled \
down the hill, with Jill following suit. \
Though slightly battered, the pair returned home to \
comforting embraces. Despite the mishap, \
their adventurous spirits remained undimmed, and they \
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions:
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
1 - Jack and Jill, siblings, go on a quest to fetch water from a hilltop well, but encounter misfortune when Jack trips on a stone and tumbles down the hill, with Jill following suit, yet they return home and remain undeterred in their adventurous spirits.

2 - Jack et Jill, frère et sœur, partent en quête d'eau d'un puits au sommet d'une colline, mais rencontrent un malheur lorsque Jack trébuche sur une pierre et dévale la colline, suivi par Jill, pourtant ils rentrent chez eux et restent déterminés dans leur esprit d'aventure.

3 - Jack, Jill

4 - {
  "french_summary": "Jack et Jill, frère et sœur, partent en quête d'eau d'un puits au sommet d'une colline, mais rencontrent un malheur lorsque Jack trébuche sur une pierre et dévale la colline, suivi par Jill, pourtant ils rentrent chez eux et restent déterminés dans leur esprit d'aventure.",
  "num_names": 2
}


#### Ask for output in a specified format

In [13]:
prompt_2 = f"""
Your task is to perform the following actions:
1 - Summarize the following text delimited by
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in Italian summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
Summary: Jack and Jill, siblings, go on a quest to fetch water from a hilltop well but encounter misfortune along the way. 
Translation: Jack et Jill, frère et sœur, partent en quête d'eau d'un puits au sommet d'une colline mais rencontrent des malheurs en chemin.
Names: Jack, Jill
Output JSON: {"french_summary": "Jack et Jill, frère et sœur, partent en quête d'eau d'un puits au sommet d'une colline mais rencontrent des malheurs en chemin.", "num_names": 2}


#### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

In [14]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is correct. They correctly identified the costs for land, solar panels, and maintenance, and calculated the total cost as a function of the number of square feet.


## Model Limitations: Hallucinations
- Boie is a real company, the product name is not real.

In [17]:
prompt = f"""
조선시대 영조에 대해서 설명해줘
"""
response = get_completion(prompt)
print(response)

조선시대 영조(영조대왕, 1724년 ~ 1776년)는 조선시대의 22대 왕으로, 정조의 아들이자 성조의 아버지입니다. 영조는 조선시대 중기에 있었던 정치적, 경제적, 문화적인 변화를 주도하였으며, 그의 통치 시기는 조선시대의 전성기로 평가되고 있습니다.

영조는 정조와 달리 국정을 안정시키고자 노력하였습니다. 그는 청나라와의 관계를 중시하며, 청나라와의 외교적인 협력을 통해 조선의 안정과 발전을 추구하였습니다. 또한, 영조는 정조 때부터 시작된 농업 개혁을 이어받아 농업 생산성을 향상시키고 국력을 강화하는 데에도 많은 노력을 기울였습니다. 이를 위해 농업 기술의 개선과 새로운 작목의 도입, 토지 개간 정책 등을 추진하였습니다.

또한, 영조는 문화 발전에도 큰 관심을 가지고 있었습니다. 그는 서적 출판과 교육을 활성화시키기 위해 노력하였으며, 특히 경국대전(경국대전은 조선시대의 대표적인 역사서)의 편찬을 주도하여 조선의 역사와 문화를 기록하였습니다. 또한, 영조는 문신들의 지원을 받아 문인들의 창작 활동을 지원하고 문화적인 교류를 촉진시키는 등 문화 발전을 위한 다양한 정책을 추진하였습니다.

하지만, 영조의 통치 시기에는 일본의 침략이 있었고, 이에 대비하기 위해 군사력을 강화하는 등 국방에도 많은 노력을 기울였습니다. 그러나 영조의 통치 시기에는 일본의 침략을 완전히 막지는 못하였으며, 이후 조선은 일본의 침략에 휩쓸리게 되었습니다.

영조는 조선시대의 중요한 왕 중 한 명으로, 그의 통치 시기는 조선시대의 전성기로 평가되고 있습니다. 그는 안정적인 정치, 경제, 문화적인 변화를 추구하였으며, 그의 노력 덕분에 조선은 상당한 발전을 이루었습니다.


#### Notes on using the OpenAI API outside of this classroom

To install the OpenAI Python library:
```
!pip install openai
```

The library needs to be configured with your account's secret key, which is available on the [website](https://platform.openai.com/account/api-keys).

You can either set it as the `OPENAI_API_KEY` environment variable before using the library:
 ```
 !export OPENAI_API_KEY='sk-...'
 ```

Or, set `openai.api_key` to its value:

```
import openai
openai.api_key = "sk-..."
```

#### A note about the backslash
- In the course, we are using a backslash `\` to make the text fit on the screen without inserting newline '\n' characters.
- GPT-3 isn't really affected whether you insert newline characters or not.  But when working with LLMs in general, you may consider whether newline characters in your prompt may affect the model's performance.